In [1]:
import graphlab

In [2]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

[INFO] This non-commercial license of GraphLab Create is assigned to 527536646@qq.com and will expire on October 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1185 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1448141986.log
[INFO] GraphLab Server Version: 1.6.1


In [3]:
graphlab.canvas.set_target('ipynb')

## Q1 summary

In [14]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


# Q2 seperate the column and store data

In [15]:
image_train_dog = image_train[image_train['label']=='dog']

In [18]:
image_train_cat = image_train[image_train['label']=='cat']

In [19]:
image_train_bird = image_train[image_train['label']=='bird']

In [20]:
image_train_automobile = image_train[image_train['label']=='automobile']

#### create a near-neigbour model

In [21]:
dog_knn_model = graphlab.nearest_neighbors.create(image_train_dog,
                                             features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [22]:
cat_knn_model = graphlab.nearest_neighbors.create(image_train_cat,
                                             features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [23]:
bird_knn_model = graphlab.nearest_neighbors.create(image_train_bird,
                                             features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [24]:
automobile_knn_model = graphlab.nearest_neighbors.create(image_train_automobile,
                                             features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


In [27]:
cat1 = image_test[0:1]

In [82]:
def get_image_from_ids(query_result,):
    return image_train_cat.filter_by(query_result['reference_label'],'id')

In [83]:
cat_neighbors = get_image_from_ids(cat_knn_model.query(cat1))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.233ms      |
PROGRESS: | Done         |         | 100         | 84.181ms     |
PROGRESS: +--------------+---------+-------------+--------------+


### nearest cat is #16289

In [87]:
cat_neighbors['image'].show()

In [90]:
image_train_cat.filter_by(cat_knn_model.query(cat1)['reference_label'][0],'id')['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 9.095ms      |
PROGRESS: | Done         |         | 100         | 82.603ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [29]:
dog_knn_model.query(cat1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 7.613ms      |
PROGRESS: | Done         |         | 100         | 82.294ms     |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


#### nearset dog like the cat is #16976

In [91]:
image_train_dog.filter_by(dog_knn_model.query(cat1)['reference_label'][0],'id')['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.115ms      |
PROGRESS: | Done         |         | 100         | 82.009ms     |
PROGRESS: +--------------+---------+-------------+--------------+


# Q3 discussing the DISTANCE

In [34]:
sum(cat_knn_model.query(cat1)['distance'][0:5])/5

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.091ms      |
PROGRESS: | Done         |         | 100         | 82.353ms     |
PROGRESS: +--------------+---------+-------------+--------------+


36.15573070978294

In [35]:
sum(dog_knn_model.query(cat1)['distance'][0:5])/5

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 7.564ms      |
PROGRESS: | Done         |         | 100         | 82.06ms      |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184157

#### we find the model for cat is coloser

# Q4 more challenge find

In [38]:
image_test_cat, image_test_dog, image_test_bird, image_test_automobile = image_test[image_test['label']=='cat'],image_test[image_test['label']=='dog'],image_test[image_test['label']=='bird'],image_test[image_test['label']=='automobile']

### find the nearest neighbors for each data point

In [42]:
dog_dog_neighbors = dog_knn_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 577.587ms    |
PROGRESS: | Done         | 509000  | 100         | 578.52ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [40]:
dog_cat_neighbors = cat_knn_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 581.735ms    |
PROGRESS: | Done         | 509000  | 100         | 582.671ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [44]:
dog_automobile_neighbors = automobile_knn_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 594.187ms    |
PROGRESS: | Done         | 509000  | 100         | 595.108ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [45]:
dog_bird_neighbors = bird_knn_model.query(image_test_dog, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 478000  | 100         | 557.921ms    |
PROGRESS: | Done         | 478000  | 100         | 558.873ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [48]:
new_sframe = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],
                              'dog-bird': dog_bird_neighbors['distance'],
                              'dog-cat': dog_cat_neighbors['distance'],
                              'dog-dog': dog_dog_neighbors['distance'],
                              })

In [49]:
new_sframe[:1]

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373


In [50]:
def is_dog_correct(row):
    if row['dog-dog']<row['dog-automobile'] and row['dog-dog']<row['dog-bird'] and row['dog-dog']<row['dog-cat']:
        return 1
    else:
        return 0

In [51]:
is_dog_correct(new_sframe[:1])

1

In [94]:
len(new_sframe)

1000

In [95]:
new_sframe.apply(is_dog_correct).sum()

678

## 67.8%

#### Then try the cat

In [53]:
cat_cat_neighbors = cat_knn_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 592.869ms    |
PROGRESS: | Done         | 509000  | 100         | 594.106ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [54]:
cat_dog_neighbors = dog_knn_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 615.748ms    |
PROGRESS: | Done         | 509000  | 100         | 616.954ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [55]:
cat_bird_neighbors = bird_knn_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 478000  | 100         | 628.424ms    |
PROGRESS: | Done         | 478000  | 100         | 629.857ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [57]:
cat_automobile_neighbors = automobile_knn_model.query(image_test_cat, k=1)

PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 1
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 509000  | 100         | 695.829ms    |
PROGRESS: | Done         | 509000  | 100         | 697.528ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [58]:
cat_sframe = graphlab.SFrame({'cat-automobile': cat_automobile_neighbors['distance'],
                              'cat-bird': cat_bird_neighbors['distance'],
                              'cat-cat': cat_cat_neighbors['distance'],
                              'cat-dog': cat_dog_neighbors['distance'],
                              })

In [59]:
def is_cat_correct(row):
    if row['cat-cat']<row['cat-automobile'] and row['cat-cat']<row['cat-bird'] and row['cat-cat']<row['cat-dog']:
        return 1
    else:
        return 0

In [60]:
cat_sframe.apply(is_cat_correct).sum()

548